## Compare with IHS Alternative Material tables: 


In [ ]:
df_altMatIHS = pd.read_csv(os.path.join(datapath, "IhsAltMat.csv"), sep = "|")
df_altMatIHS.sort_values(by = "objectId", inplace=True)
#df_altMatIHS = df_altMatIHS[["objectId","partNumber","alternativeObjectId"]]

In [ ]:
df_altMatIHS["partNumber"] = df_altMatIHS["partNumber"].str.replace('\W+', '').str.strip()
df_altMatIHS["AltpartNumber"] = df_altMatIHS["AltpartNumber"].str.replace('\W+', '').str.strip()

In [ ]:
check = df_altMatIHS["AltpartNumber"].isin(df_sim_map.SimilarPartNumber.tolist())

In [ ]:
df_altMatIHS.drop_duplicates(inplace=True)


In [ ]:
df = df_altMatIHS[["partNumber", "AltpartNumber"]]
df.drop_duplicates(inplace=True)
df.shape

In [ ]:
df_sim_check = df_sim_map.merge(df_altMatIHS,  left_on = ["KeyPartNumber","SimilarPartNumber"], right_on = ["partNumber", "AltpartNumber"])
print(len(df_sim_check) / len(df_altMatIHS) )

In [ ]:
df_sim_check.to_csv(os.path.join(datapath, "Check_with_AltIHS.csv"))

In [ ]:
df_sim_map = df_sim_map.merge(df_altMatIHS[["partNumber", "AltpartNumber"]], how = "left", left_on = ["KeyPartNumber","SimilarPartNumber"], right_on = ["partNumber", "AltpartNumber"])
df_sim_map.shape

In [ ]:
df_sim_map.head()

In [ ]:
def drop_duplicate_cols(df): 
    """ returns df with duplicate columns removed"""
    return df.loc[:,~df.columns.duplicated()]

### Which pairs did the algo missed?


In [ ]:
df_sim_map.loc[df_sim_map.KeyPartNumber.str.contains("933215370235")].head(30)

In [ ]:
df_missed = df_altMatIHS.merge(df_sim_map,how = "left", right_on = ["KeyPartNumber","SimilarPartNumber"], left_on = ["partNumber", "AltpartNumber"])
df_missed.loc[df_missed.KeyPartNumber.isna() == True].head(20)

In [ ]:
df_tmp = df_ABT.loc[df_ABT.index.str.contains("SI4850EY") == True]
df_tmp.dropna(axis = 1, how = "all", inplace=True)
df_tmp.head()

In [ ]:
#check = df_ABT.index.str.contains("BZV55C8V2")
np.where(check == True)

In [ ]:
X = dict_X["IHS"]

In [ ]:
tmp = (X[26330,] * X.T).toarray().flatten()
cos_dist = np.round(tmp / np.max(tmp),4)
cos_dist

In [ ]:
threshold = 0.8
n_neighbors = 100
most_sim_n_neighbors = np.argsort(tmp / np.max(tmp))[::-1][:n_neighbors]
#most_sim_IHS = [idx for idx in most_sim_n_neighbors if cos_dist[idx] > threshold]
    
most_sim = [idx for idx in most_sim_n_neighbors if cos_dist[idx] > threshold]

In [ ]:
np.where(most_sim == 26333)

In [ ]:
df_sim_map.loc[df_sim_map.KeyPartNumber.str.contains("BZV55C8V2") == True].head(20)

## Define similarity pipelines for overall, form fit  & function

In [ ]:
df_tmp = df_ABT.loc[df_ABT.index.str.contains("BZV55C8V2|BZV55B13")].dropna(axis = 1, how = "all")

In [ ]:
df_tmp.to_csv(os.path.join(datapath, "Example_BZV55C8V2.csv"))

In [ ]:
df_sim_map["objectId"] = df_sim_map.objectId_x.astype(str)
df_sim_map["commId"] = df_sim_map.commId.astype(str)

In [ ]:
df_sim_map.drop(columns="objectId_x", inplace=True)

In [ ]:
df_sim_map.to_csv(os.path.join(datapath, "SimMap.csv"), index = False)
#df_master.to_csv(os.path.join(datapath, "ABT_IHS.csv"), index = False)

In [ ]:
store.open()
df_test = store["SimMap"]
df_test.head()

In [ ]:
cols_to_consider = [cols for cols in cols_to_consider if cols not in ("Description_IHS", "partType", "category", "text_cleaned","Sim_measure")]
df_similarityMap.drop(columns= cols_to_consider, inplace=True)
#len(df_similarityMap.columns)
#len(data_final.columns)
#len(cols_to_consider)

In [ ]:
df_similarityMap.head()

In [ ]:
import sys
print("Number of rows of data: " , len(df_similarityMap))
print("Storage in GB used: ", sys.getsizeof(df_similarityMap) / 1e9)

In [ ]:
print(datapath)

In [ ]:
df_reduced = df_similarityMap.head(10000)
#print("Storage in GB used: ", sys.getsizeof(df_reduced) / 1e9)
df_reduced.to_csv(os.path.join(datapath, "SimilarityResistors_First10000.csv"), encoding = "utf-8")

## Store into the database

In [ ]:
con_to_store = pass

df_similarityMap.to_sql(con=con_to_store, name='SimilarityMap_resistors', if_exists='replace', flavor='mysql')

## Descriptive Analysis of the different attributes

In [ ]:
for key, value in att_dict["capacitors"].items():
    if key == 'type_all':
        continue
    print(value)

In [ ]:
df_relay = df_ABT['Product Length'].loc[df_ABT.Cat.str.contains('rela')]

In [ ]:
df_ABT[df_ABT.index.str.contains("1-1393210-3")]

In [ ]:
df_relay.to_csv(os.path.join(datapath, "relay_prod_length.csv"))

In [ ]:
result_list = list()
for cat in att_dict.keys():
    if cat != 'other':
        print(cat)
        for key, value in att_dict[cat].items():

            if key == 'type_all':
                continue
            if len(value) > 0:
                df_tmp = df_per_cat[cat][value].describe(include = 'all')
                df_transpose = df_tmp.T
                df_transpose["Category"] = np.repeat([cat], df_transpose.shape[0])
                df_transpose["Type"] = np.repeat([key], df_transpose.shape[0])

                result_list.append(df_transpose)
    else: 
        print(cat)
        df_tmp = df_per_cat[cat].describe(include = 'all')
        df_transpose = df_tmp.T
        df_transpose["Category"] = np.repeat([cat], df_transpose.shape[0])
        df_transpose["Type"] = np.repeat(['All'], df_transpose.shape[0])

        result_list.append(df_transpose)

In [ ]:
df_descriptive = pd.concat(result_list)

In [ ]:
df_descriptive.to_csv(os.path.join(datapath, "DescriptiveAnalysis.csv"))

In [ ]:
df_tmp = df_per_cat[cat].describe(include = 'all')
df_tmp["Category"] = np.repeat([cat], df_tmp.shape[0])
df_tmp

In [ ]:
att_stats = dict()
for cat in att_dict.keys():
    print(cat)
    if cat != 'other':
        att_stats[cat] = pd.concat([df_ABT[att_dict[cat][mode_]].describe(include = "all") for mode_ in [elem for elem in mode if elem != 'type_all'] if len(att_dict[cat][mode_]) > 0]
                                      ,keys = [elem for elem in mode if elem != 'type_all'])
    else:         
        att_stats[cat] = df_ABT[att_dict[cat]['type_all']].describe(include = "all")

In [ ]:
df_tmp = att_stats['capacitors'].copy()

In [ ]:
df_tmp2 = df_tmp.dropna(axis = 0, how = 'all')
df_tmp2

In [ ]:
for mode_ in [elem for elem in mode if elem != 'type_all']: 
    